## RadPatterns Usage Demonstration

written by Jessica Hawthorne, July 2025

In [2]:
import numpy as np
import radpatterns

### 1. Background

The codes here are written to calculate the seismic radiation coefficients of P, SV, and SH waves from a point source.  You specify the fault's strike and dip, along with the rake amount of tensile opening.  You also specify the takeoff angle and azimuth of the outgoing seismic wave.  The code then calculates the amplitude and signs of the radiated wave.

The physics and equations underlying these calculations are documented in a number of textbooks and papers, including

Madariaga, R. (2015). Seismic source: Theory . In: _Geophysics. Encyclopedia of Earth Science._ Springer, Boston, MA, 10.1016/B978-0-444-53802-4.00070-1.
                                                                                
Ou, G. B., 2008: Seismological studies for tensile faults. _Terr. Atmos. Ocean. Sci._, 19, 463-471, 10.3319/TAO.2008.19.5.463(T).                           

To summarise,  the far-field P, SH, and SV waves in a homogeneous medium can be written as scaled, delayed versions of the moment rate function $\dot{M}$.  Following Madariaga (2015, equation 21), the particle motions are 

${\bf u}^P(t)  = \Re^P \frac{1}{4\pi \rho \alpha^2} \frac{1}{R}  \dot{M}(t-R/\alpha) {\bf e}_R$

${\bf u}^{SH}(t)  = \Re^{SH} \frac{1}{4\pi \rho \beta^2} \frac{1}{R}  \dot{M}(t-R/\beta) {\bf e}_{\phi}$

${\bf u}^{SV}(t)  = \Re^{SV} \frac{1}{4\pi \rho \beta^2} \frac{1}{R}  \dot{M}(t-R/\beta) {\bf e}_{\theta}$

This code calculates the radiation coefficients $\Re^P$, $\Re^{SH}$, and $\Re^{SV}$.  

$\rho$ is the material density, $\alpha$ is the P wave velocity, $\beta$ is the S wave velcoity, and $R$ is the distance from the source to receiver.

The unit vectors ${\bf e}_R$, ${\bf e}_\phi$, and ${\bf e}_\theta$ indicate the particle motion directions.  Here we have chosen ${\bf e}_R$ to point directly away from the source, along the raypath.  ${\bf e}_\phi$ is oriented in a horizontal plane, perpendicular to the raypath, with positive to the right when facing the direction of travel.  ${\bf e}_\theta$ is oriented in a vertical plane, perpendicular to the raypath, with positive having a positive component in the upward direction.  

### 2. Calculating a moment tensor

You may wish to calculate a moment tensor given a strike, dip, rake, and fault opening.  To do so, use the function create_moment_tensor.  

To calculate the moment tensor for a double couple, just specify the strike, dip, and rake.

In [5]:
# say we want an vertical E-W striking right-lateral strike-slip fault
M=radpatterns.create_moment_tensor(strike=90,dip=90,rake=180)

print(np.round(M,2))

[[-0.  1. -0.]
 [ 1.  0. -0.]
 [-0. -0.  0.]]


The moment tensor can also allow some tensile motion, or opening of the fault.  To do so, specify the opening angle: the direction of the surface of the fault relative to the fault plane. An opening angle of 0 gives pure shear slip.  An opening angle of 90 gives pure tensile motion.

Note that opening particle motion creates stresses in all three directions.  To model these stresses, we must specify the material's Poisson's ratio.  By default, Poisson's ratio is taken to be 0.25.

In [6]:
# say you want to calculate a moment tensor for tensile fault of a horizontal fault
# the strike and rake will be irrelevant here
M=radpatterns.create_moment_tensor(strike=0,dip=0,rake=0,opening_angle=90,poissons_ratio=0.25)

print(np.round(M,2))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 3.]]


### 3. Calculate the radiation coefficients

Given a moment tensor, you may wish to calculate the radiation coefficients $\Re^P$, $\Re^{SH}$, and $\Re^{SV}$.  To do so, use the function calc_radiation_pattern.  You must specify the azimuth and takeoff angle of the outgoing wave.  The azimuth is measured in degrees east of north.  The takeoff angle is measured in degrees from down.

In [8]:
# let's go back to our E-W right-lateral strike-slip fault
M=radpatterns.create_moment_tensor(strike=90,dip=90,rake=180)

# and look at a nearby station to the NE
rd_p,rd_sh,rd_sv = radpatterns.calc_radiation_pattern(M,takeoff_angle=130,azimuth=45)
print('P radiation coefficient nearby to the NE: {:0.2f}'.format(rd_p))

# or a more distant station to the NW
rd_p,rd_sh,rd_sv = radpatterns.calc_radiation_pattern(M,takeoff_angle=20,azimuth=-45)
print('P radiation coefficient far away to the NW: {:0.2f}'.format(rd_p))

P radiation coefficient nearby to the NE: 0.59
P radiation coefficient far away to the NW: -0.12


### 4. Other functions  

Two other functions are available.  

The function "fault_projected_moment_tensor" calculates a shear _or_ tensile moment tensor, but returns it in a coordinate system with axes defined by the fault plane.

The function "calc_unit_vectors" calculates the unit vectors ${\bf e}_R$, ${\bf e}_\phi$, and ${\bf e}_\theta$: the assumed directions of particle motion for the P, SH, and SV waves.  Note that here we have used the coordinate system described above, with positive to the right and up for the SH and SV waves, respectively.  Some references use different definitions of positive and negative.